In [1]:
# Import dependencies
from time import sleep
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183
[WDM] - Driver [C:\Users\Boss\.wdm\drivers\chromedriver\win32\85.0.4183.87\chromedriver.exe] found in cache


## Extract: scrape recipe data from bbc.co.uk/food

In [3]:
url = "https://www.bbc.co.uk/food/cuisines"

# Use splinter to navigate the site
browser.visit(url)

In [4]:
# Create BeautifulSoup object; parse with 'lxml'
html = browser.html
soup = bs(html, 'lxml')

In [5]:
# Retrieve all text with links to each cuisine
cuisines = [cuisine.get_text() for cuisine in soup.find_all("h3", class_ = 'promo__title gel-pica')]
cuisines

['African',
 'American',
 'British',
 'Caribbean',
 'Chinese',
 'East European',
 'French',
 'Greek',
 'Indian',
 'Irish',
 'Italian',
 'Japanese',
 'Korean',
 'Mexican',
 'Nordic',
 'North African',
 'Pakistani',
 'Portuguese',
 'South American',
 'Spanish',
 'Thai and South-East Asian',
 'Turkish and Middle Eastern']

In [6]:
cuisine_ingredients = list()

for cuisine in cuisines:
    
    # Click the links to all recipes of each cuisine.
    browser.links.find_by_partial_text(cuisine).click()
    sleep(1)
    browser.links.find_by_partial_text(f"all {cuisine} recipes").click()
    sleep(1)
    html = browser.html
    soup = bs(html, 'lxml')
    
    # Retrieve all recipe names in the current page
    recipes = [recipe.get_text() for recipe in soup.find_all("h3", class_ = "promo__title")]

    for recipe in recipes:
        browser.links.find_by_partial_text(recipe).click()
        sleep(1)
        html = browser.html
        soup = bs(html, 'lxml')

        # Retrieve list of ingredients
        ingredients = [ingredient.get_text() for ingredient in soup.find_all(
            "a", class_ = "recipe-ingredients__link")]
        full_ingredients = [ingredient.get_text() for ingredient in soup.find_all(
            "li", class_ = "recipe-ingredients__list-item")]
        # Retrieve recipe image if available
        try:
            recipe_image = soup.select_one("div.recipe-media__image img")["src"]
        except TypeError:
            recipe_image = None
            
        # Append a dictionary with the above info to a list
        cuisine_ingredients.append({
            "cuisine": cuisine,
            "recipe": recipe,
            "ingredients": ingredients,
            "full_ingredients": full_ingredients,
            "recipe_image": recipe_image
        })
            
        browser.back()
        sleep(1)
        
    html = browser.html
    soup = bs(html, 'lxml')
    
    # Click the next recipe page in navigation bar down to the bottom of the page (if available)
    try:
        max_page = int(soup.find_all("li", class_="pagination__list-item")[-2].get_text())

        for i in range(2, max_page+1, 1):
            if i == 2 or i == max_page:
                browser.find_by_xpath(
                    f'//*[@id="main-content"]/div[1]/div[3]/div/div[6]/div[4]/ul/li[{max_page+1}]/a').click()
            else:
                browser.find_by_xpath(
                    f'//*[@id="main-content"]/div[1]/div[3]/div/div[6]/div[4]/ul/li[{max_page+2}]/a').click()

            sleep(1)
            html = browser.html
            soup = bs(html, 'lxml')

            # Retrieve all recipes in the current page
            recipes = [recipe.get_text() for recipe in soup.find_all("h3", class_ = "promo__title")]

            for recipe in recipes:
                browser.links.find_by_partial_text(recipe).click()
                sleep(1)
                html = browser.html
                soup = bs(html, 'lxml')

                # Retrieve list of ingredients
                ingredients = [ingredient.get_text() for ingredient in soup.find_all(
                    "a", class_ = "recipe-ingredients__link")]
                full_ingredients = [ingredient.get_text() for ingredient in soup.find_all(
                    "li", class_ = "recipe-ingredients__list-item")]
                
                # Retrieve recipe image if available
                try:
                    recipe_image = soup.select_one("div.recipe-media__image img")["src"]
                except TypeError:
                    recipe_image = None
            
                # Append a dictionary with the above info to a list
                cuisine_ingredients.append({
                    "cuisine": cuisine,
                    "recipe": recipe,
                    "ingredients": ingredients,
                    "full_ingredients": full_ingredients,
                    "recipe_image": recipe_image
                })
                
                browser.back()
                sleep(1)

        # Back to the starting url
        browser.visit(url)
        sleep(1)

    except IndexError:
        # Back to the starting url
        browser.visit(url)
        sleep(1)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=85.0.4183.121)


In [ ]:
khi nao co loi thi in loi thi append vao 1 csv rieng va link cua trang bi loi
append vao csv

In [ ]:
# Quit the browser after scraping
browser.quit()

## Consolidate data scraped from zoo.org.au and environment.vic.gov.au

In [ ]:
# Overview of data scraped from environment.vic.gov.au
animal_info_image_environment_df = pd.DataFrame(animal_info_image_environment)
animal_info_image_environment_df.head()

In [ ]:
# Export to csv for merging against the vba fauna data
final_animal_image_df.to_csv("../data/animal_image_to_merge.csv", index=False)